In [23]:
import pandas as pd
import random
import os
import numpy as np
import pickle

# load word vecs

In [2]:
data_path = r"C:\Users\alexa\Documents\data\wir\project"
fname = r"cc.en.300.bin"

In [3]:
%%time
import fasttext
model = fasttext.load_model(
    os.path.join(data_path, fname))

Wall time: 1min 31s


In [14]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = map(float, tokens[1:])
    return data

In [15]:
data_path = r"C:\Users\alexa\Documents\data\wir\project"
fname = r"wiki-news-300d-1M-subword.vec"

In [ ]:
%time
wv = load_vectors(
    os.path.join(data_path, fname))

In [4]:
import csv

In [5]:
%%time
data_path = r"C:\Users\alexa\Documents\data\nlp\nlp_hw3"
glove_data_file = 'glove.6B.300d.txt'
emb = pd.read_table(os.path.join(data_path, glove_data_file), sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

Wall time: 52.9 s


# generate img embeddings

In [477]:
from imgsearch import cosine_sim

def getwv(s):
    return model.get_word_vector(s.lower())

def getwv_multi_labels(labels):
    #sublabels = labels.split(', ')
    sublabels = re.split(r'[, ]\s*', labels)
    return sum([getwv(subl) for subl in  sublabels]) / len(sublabels)

In [478]:
# img emb = weighted sum of vec
get_weighted_sum_vec = lambda tup: sum([getwv(label) * weight for weight, label in tup])
get_weighted_sum_vec_multilbl = lambda tup: sum([getwv_multi_labels(label) * weight for weight, label in tup])

# img emb = [(vec, weight)]
get_tup_vec_weigth = lambda tup: [(getwv(label), weight) for weight, label in tup]
get_tup_vec_weight_multilbl = lambda tup: [(getwv_multi_labels(label), weight) for weight, label in tup]

# img emb = vec of first label
get_vec_first_label = lambda tup: sum([getwv_multi_labels(label) for weight, label in [tup[0]]])

In [479]:
path='static/img_top10_labels_w_weights.csv'
    
# load image label data
labelled_df = pd.read_csv(path)

In [480]:
from ast import literal_eval
labelled_df.content = labelled_df.content.apply(literal_eval)

In [481]:
%%time
img_emb_dict = dict(zip(
    labelled_df.img_id,
    #labelled_df.content.apply(get_weight_vec_multilbl) # img emb = weighted sum of vec
    #labelled_df.content.apply(get_vec_first_label) # mg emb = vec of first label
    labelled_df.content.apply(get_tup_vec_weight_multilbl)# img emb = [(vec, weight)]
))

Wall time: 1.96 s


In [482]:
import pickle
#with open(r"static/img_emb_dict.pkl", "wb") as f:
#    pickle.dump(img_emb_dict, f)

# build search routine

In [483]:
#with open(r"static/img_emb_dict.pkl", "rb") as f:
#    img_emb_dict = pickle.load(f)

In [484]:
def image_search_res5(query, img_emb_dict):
    
    # get query embedding representation
    q = getwv_multi_labels(query)
    
    if q.sum() != 0:
        
        # parse query
        # ...
        
        # map image ids to names
        img_id_to_name = dict(zip(labelled_df.img_id, labelled_df.image_name))
        
        # score and rank images
        img_idxs = img_emb_dict.keys()
        score = [(
            img_id_to_name[key]
            #,cosine_sim(q, img_emb_dict[key]) # img emb = weighted sum of vec
            ,sum([weight * cosine_sim(q, vec) for vec, weight in img_emb_dict[key]]) # img emb = [(vec, weight)]
            #,sum([weight * cosine_sim(q, vec) for vec, weight in img_emb_dict[key]][:3]) # img emb = [(vec, 1)]
        ) for key in img_idxs]
        ranked = sorted(score, key=lambda tup: tup[1], reverse=True)
    
        # output list of images and proba
        return ranked[:12]
    else:
        return []

### inspect single image and assess relevance with query

In [485]:
def inspect_img_vs_query(query, img_id):
    print(
        labelled_df.query("img_id == '{}'".format(img_id)).image_name.values[0])
    scores = [(weight, cosine_sim(getwv_multi_labels(query), vec)) for vec, weight in img_emb_dict['{}'.format(img_id)][:3]]
    scores = [(weight * sim, weight, sim) for weight, sim in scores]
    print(
        sum([score[0] for score in scores]))
    print(
        labelled_df.query("img_id == '{}'".format(img_id)).loc[:,'content'].values[0][:3])
    return scores

In [516]:
cosine_sim(getwv_multi_labels("plunger, plumber's helper"), getwv_multi_labels("asdfsgd"))

0.26644495

In [494]:
inspect_img_vs_query('asdfsgd', 7190)

842-7190-large_default-plume-de-bureau.jpg
0.21230353115817727
((0.7355284094810486, "plunger, plumber's helper"), (0.1740245223045349, 'table lamp'), (0.01282315794378519, 'spindle'))


[(0.19597783124089396, 0.7355284094810486, 0.26644495),
 (0.014060712787803187, 0.1740245223045349, 0.08079731),
 (0.002264987129480117, 0.01282315794378519, 0.17663255)]

In [523]:
inspect_img_vs_query('bicyle', 13437)

1621-13437-large_default-organisateur-high-tech-bellroy.jpg
0.29762976293333027
((0.9043031930923462, 'seat belt, seatbelt'), (0.03341376408934593, 'scabbard'), (0.018360324203968048, 'buckle'))


[(0.2875137230397229, 0.9043031930923462, 0.31793952),
 (0.0065252400276893385, 0.03341376408934593, 0.19528599),
 (0.0035907998659180373, 0.018360324203968048, 0.19557388)]

### query images and save output in static folder

In [520]:
query = 'book'
image_list = image_search_res5(query, img_emb_dict)
with open(r"static/image_list.pkl", "wb") as f:
    pickle.dump(image_list, f)
image_list

[('965-8361-large_default-un-rendez-vous-chez-le-barbier.jpg',
  0.5948124769326235),
 ('1797-14438-large_default-casque-de-velo-urbain.jpg', 0.49601409581487826),
 ('1797-14434-large_default-casque-de-velo-urbain.jpg', 0.41289129330575647),
 ('1797-14411-large_default-casque-de-velo-urbain.jpg', 0.32578944563839196),
 ('1607-13245-large_default-gants-en-cuir-double-cachemire-by-dents.jpg',
  0.325553129840431),
 ('626-5109-large_default-adopter-vos-pieds-de-vigne.jpg', 0.3245709244848521),
 ('1797-14421-large_default-casque-de-velo-urbain.jpg', 0.324302362326393),
 ('1797-14413-large_default-casque-de-velo-urbain.jpg', 0.32354706392542115),
 ('1797-14418-large_default-casque-de-velo-urbain.jpg', 0.32161969861347534),
 ('1797-14412-large_default-casque-de-velo-urbain.jpg', 0.31081113567197055),
 ('1621-13437-large_default-organisateur-high-tech-bellroy.jpg',
  0.30456713148728515),
 ('965-8359-large_default-un-rendez-vous-chez-le-barbier.jpg',
  0.2930161408907653)]

# add weights to image labels

In [242]:
from sklearn.feature_extraction.text import CountVectorizer
from imgsearch import cosine_sim
import numpy as np

In [9]:
#pd.read_csv('static/imagenet_classes_to_labels_map.txt')
with open('static/imagenet_classes_to_labels_map.txt') as f:
    labels = eval(f.read())

In [10]:
def get_topn_labels(row, n=10):
    outdf = row.astype(float).nlargest(n).reset_index()
    outdf.columns = ['label_id','percentage']
    outdf.label_id = outdf.label_id.astype(int)
    outdf['label_name'] = outdf.label_id.map(labels)
    return outdf

In [11]:
%%time

n_img = 10000

# get weight data
data_path = r"C:\Users\alexa\Documents\data\wir\project\img_proba_vecs_vgg19.csv"
dfw = pd.read_csv(data_path, low_memory=False)

# generate df with descr of top10 classes
labelled_df = dfw.loc[:, dfw.columns[:3]].copy()
labelled_df['content'] = ''

for i, image_name in enumerate(dfw.image_name.loc[:n_img]):
    topnlabels = get_topn_labels(dfw.loc[i, dfw.columns[3:]])
    labelled_df.loc[i, 'content'] = tuple(zip(topnlabels.percentage, topnlabels.label_name)) # keep weight info

# select top n images
labelled_df = labelled_df.loc[:n_img].copy()

Wall time: 1min 11s


In [12]:
    path='static/img_top10_labels_w_weights.csv'
    
    # load image label data
    labelled_df.to_csv(path, index=False)